In [4]:
using FermiCG
using PyCall
using Plots
using LinearAlgebra
using Printf

## Import PYSCF data

In [5]:
pyscf = pyimport("pyscf");
fcidump = pyimport("pyscf.tools.fcidump");
ctx = fcidump.read("fe2s2");
h = ctx["H1"];
g = ctx["H2"];
ecore = ctx["ECORE"];
g = pyscf.ao2mo.restore("1", g, size(h,2));

In [6]:
# blocking from paper
blocks = [[0,1],[2,3,4,5,6],[7,9,11],[8,10,12],[13,14,15,16,17],[18,19]]
init_fspace = ((2,2),(5,0),(3,3),(3,3),(5,0),(2,2))

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)

6-element Array{Cluster,1}:
 Cluster(1, [1, 2])
 Cluster(2, [3, 4, 5, 6, 7])
 Cluster(3, [8, 10, 12])
 Cluster(4, [9, 11, 13])
 Cluster(5, [14, 15, 16, 17, 18])
 Cluster(6, [19, 20])

In [ ]:
# blocking 1
blocks = [  [0,1,2,3,6,7,8],        # L1(sig)+ Fe1(t) + L2(sig)
            [11,12,13,16,17,18,19], # L2(sig)+ Fe2(t) + L3(sig)
            [4,5,9,10,14,15],       # Fe1(e) + Fe2(e) + L2(pi)
         ]

init_fspace = ((7,4), (7,4), (4,4))

for b in blocks
    b .+= 1
end
clusters = [Cluster(i,collect(blocks[i])) for i = 1:length(blocks)]
display(clusters)


In [7]:
ints = InCoreInts(ecore,h,g);

na = 0
nb = 0
for f in init_fspace
    na += f[1]
    nb += f[2]
end
display(na)
display(nb)
# get initial guess for 1RDM
tmp = eigen(ints.h1).vectors;
rdm1a_guess = tmp[:,1:na]*tmp[:,1:na]';
rdm1b_guess = tmp[:,1:nb]*tmp[:,1:nb]';
# eigen(ints.h1).values


20

10

In [10]:
e_cmf, U, Da, Db  = FermiCG.cmf_oo(ints, clusters, init_fspace, rdm1a_guess, rdm1b_guess,
                                        max_iter_oo=40, verbose=0, gconv=1e-9, method="bfgs");
ints2 = FermiCG.orbital_rotation(ints,U);


 ooCMF Iter:    1 Total= -115.958971122269 Active= -115.958971122269 G=     5.68e-01
 ooCMF Iter:    2 Total= -116.452566166470 Active= -116.452566166470 G=     2.75e-01
 ooCMF Iter:    3 Total= -116.487390854933 Active= -116.487390854933 G=     1.43e-01
 ooCMF Iter:    4 Total= -116.503317395887 Active= -116.503317395887 G=     6.98e-02
 ooCMF Iter:    5 Total= -116.506555919580 Active= -116.506555919580 G=     2.48e-02
 ooCMF Iter:    6 Total= -116.506990651447 Active= -116.506990651447 G=     8.78e-03
 ooCMF Iter:    7 Total= -116.507077222629 Active= -116.507077222629 G=     6.40e-03
 ooCMF Iter:    8 Total= -116.507125519400 Active= -116.507125519400 G=     2.62e-03
 ooCMF Iter:    9 Total= -116.507130990476 Active= -116.507130990476 G=     1.05e-03
 ooCMF Iter:   10 Total= -116.507131947301 Active= -116.507131947301 G=     5.05e-04
 ooCMF Iter:   11 Total= -116.507132184537 Active= -116.507132184537 G=     2.42e-04
 ooCMF Iter:   12 Total= -116.507132234567 Active= -116.507132234

 * Status: success

 * Candidate solution
    Final objective value:     -1.165071e+02

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 4.20e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.10e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.84e-14 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.44e-16 ≰ 0.0e+00
    |g(x)|                 = 9.62e-10 ≤ 1.0e-09

 * Work counters
    Seconds run:   4  (vs limit Inf)
    Iterations:    25
    f(x) calls:    75
    ∇f(x) calls:   75


 ooCMF Iter:   26 Total= -116.507132250720 Active= -116.507132250720 G=     2.66e-09
*ooCMF -116.50713225 
